## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

C:\Users\kajal\miniconda3\envs\pytorch\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


## Create Fully Connected Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):   #(28*28)MNIST image =784 is input size, 
        super(NN, self).__init__()   #super calls initialize method of parent class(i.e, nn.Module) 
        # Lets create a network()simple 2 layer
        # nn.Linear(in_Features, out_features )
        self.fc1 = nn.Linear(input_size, 50)    
        self.fc2 = nn.Linear(50, num_classes)     #hidden nodes
        
    # define forward method, on some input x to perform actions on layers fc1, fc2 we defined
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# #check: if it runs and gives correct output on some random generated data
# #initialze model
# model = NN(784, 10)   #input size, and num_classes=10(num of digits in mnist)
# x = torch.randn(64, 784)  #initialze x, 64:num of examples will run simultaneously
# print(model(x).shape)  #should get 64*10

## Set Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [5]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

## Load Data

In [6]:
import os
work_dir = os.getcwd()
work_dir

'D:\\MyStudy\\pytorch_tutorial'

In [7]:
#where to store dataset:your cwd -> make folder 'dataset'
#When it loads the data its going to be numpy array, so transform to tensor to run in pytorch
#download = True : if dataset is exist in current folder, don't download, else download
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = 'dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle = True)

## Initialize Network

In [8]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

## Loss and Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()       #LOss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)   #optimizer

## Train Network

In [10]:
## one epoch : network has seen all the images in the dataset
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):   #goes through each batch in train loader
        #enumerate : to see which batch index we have
        #(data, target) : tuple, data:input images, target:correct digits for each label
        #Get data to cuda if possible
        data = data.to(device=device)#make the data which is currently in tensor to the device that we are using
        targets = targets.to(device=device)
        
        #print(data.shape)
        ## ([64,1,28,28]) : Number of input images, number of channel(as its black and white), height8width
        ## We want 28*28=784 vector, hence need to unroll this matrix to vector
        ## Get to correct shape
        data = data.reshape(data.shape[0], -1)
        data.shape
        
        # Forward part of NN
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backword propagation
        optimizer.zero_grad()    # so that it doesn't store the back calculations of previous forward pass
        loss.backward()
        
        # Gradient descent or adama stem
        optimizer.step()

## Check accuracy on training & test to see how good our model

In [11]:
def check_accuracy(loader, model):      #import loader and the model
    if loader.dataset.train:
        print('checking accuracy on training data')
    else:
        print('checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)          #can set 0/1 : as max score will of 2D
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)      #64
        
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 59058/60000 with accuracy 98.43
checking accuracy on test data
Got 9706/10000 with accuracy 97.06
